In [1]:
#import
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import scipy.io as sio
import os
import h5py
import datetime
from datetime import *
import scipy.stats as scs
from general_function import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
import copy
myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
from matplotlib.font_manager import FontProperties
#myfont = FontProperties(fname='/Library/Fonts/Songti.ttc')
sns.set(
    font=myfont.get_name(),
    style="whitegrid",
    palette="muted",
    color_codes=True)

In [2]:
#data
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
zzIcData = pd.read_csv('zz500_from_db.csv')
zzIcData.columns = [int(i) for i in zzIcData.columns]
zzIcData[1] = [int(i) for i in zzIcData[1]]
zzIcData.set_index(1,inplace=True)
full = pd.read_csv('full_data_for_generator_from_db.csv')
full.columns = [int(i) for i in full.columns]
full[0] = [int(i) for i in full[0]]
full[4] = [int(i) for i in full[4]]
full.set_index([0,4],inplace=True)
os.chdir(r'F:\金湖回测系统\股票数据')
STcode_by_date = pd.read_csv('STcode_by_date.csv',index_col=0)
STcode_by_date = STcode_by_date[STcode_by_date.index>=20170101]
def process_co(df):
    return df[5]/df[8].shift(1)-1
co = full.groupby(level=0).apply(process_co)
co.index = co.index.droplevel(0)
co.name = 0
def process_oc(df):
    return df[8]/df[5]-1
oc = full.groupby(level=0).apply(process_oc)
oc.index = oc.index.droplevel(0)
oc.name = 0
def process_cc(df):
    return df[8]/df[8].shift(1)-1
cc = full.groupby(level=0).apply(process_cc)
cc.index = cc.index.droplevel(0)
cc.name = 0
def process_ch(df):
    return df[6]/df[8].shift(1)-1
ch = full.groupby(level=0).apply(process_ch)
ch.index = ch.index.droplevel(0)
ch.name = 0
def process_cl(df):
    return df[7]/df[8].shift(1)-1
cl = full.groupby(level=0).apply(process_cl)
cl.index = cl.index.droplevel(0)
cl.name = 0
def process_amount(df):
    return df[10]
amount = full.groupby(level=0).apply(process_amount)
amount.index = amount.index.droplevel(0)
amount.name = 0
def process_amount(df):
    return df[10].shift(1)
shifted_amount = full.groupby(level=0).apply(process_amount)
shifted_amount.index = shifted_amount.index.droplevel(0)
shifted_amount.name = 0
def process_amount(df):
    return df[10].shift(2)
shifted_amount_1 = full.groupby(level=0).apply(process_amount)
shifted_amount_1.index = shifted_amount_1.index.droplevel(0)
shifted_amount_1.name = 0
def process_amount(df):
    return df[10].shift(3)
shifted_amount_2 = full.groupby(level=0).apply(process_amount)
shifted_amount_2.index = shifted_amount_2.index.droplevel(0)
shifted_amount_2.name = 0
is_trade = amount.copy()
is_trade.fillna(0,inplace = True)
is_trade[is_trade!=0]=2
is_trade[is_trade==0]=1
is_trade[is_trade==2]=0
mid = STcode_by_date.stack().copy().reset_index()
mid['level_0'] = [date.toordinal(datetime(i//10000,i%10000//100,i%100))+366 for i in mid['level_0']]
mid[0] = mid[0].astype(int)
mid['level_1'] = 1
mid.columns = [4,None,0]
mid = mid.set_index([0,4])[None]
is_st = pd.Series(0,index=is_trade.index)
is_st[is_st.index.isin(mid.index)] = 1
is_dieting = pd.Series(0,index=is_trade.index)
is_dieting[is_dieting.index.isin(ch[ch<=-0.099].index)] = 1
is_dieting[is_dieting.index.isin(ch[ch.index.isin(is_st[is_st==1].index)][ch[ch.index.isin(is_st[is_st==1].index)]<-0.0495].index)] = 1

In [3]:
#preprocess *.mat
os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
filenames = listdir(file_type='.mat')
filenames_ = [[i.split('.\\')[-1].split('_')[0],i[5:9],0] if '万' not in i else \
              [i.split('.\\')[-1].split('_')[0],i[5:9],i.split('万')[0][-4:]] for i in filenames]
def process(df):
    return df.index.tolist()
by_model = pd.DataFrame(filenames_).groupby([0,2]).apply(process)

In [4]:
full_cont = {}
for (model,limit) in by_model.index:
    if model in ['A2','A3','B4','C3','E2','F1']:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model)[::-1]:
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            file = sio.loadmat(name)
            date_in_this_file = [i[0][0] for i in file['table'][1]]
            if name == sorted(filenames_model)[0] or date_in_this_file[0] >= 737392:
                new = [i for i in date_in_this_file if i not in already_date][::-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            else:
                new = [i for i in date_in_this_file if i not in already_date][::-1][:-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1][:-1]
            old_length = len(already_date)
            already_date += new
            new_length = len(already_date)
            for i in new_index:
                already_rank.append([j[0] for j in file['table'][0][i]])
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        full_cont[model] = cont
    else:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model)[::-1]:
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            file = sio.loadmat(name)
            date_in_this_file = [i[0][0] for i in file['table'][1]]
            new = [i for i in date_in_this_file if i not in already_date][::-1]
            new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            old_length = len(already_date)
            already_date += new
            new_length = len(already_date)
            for i in new_index:
                already_rank.append([j[0] for j in file['table'][0][i]])
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        full_cont[model] = cont
full_cont_before = {}
for (model,limit) in by_model.index:
    if model in ['A2','A3','B4','C3','E2','F1']:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model):
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            file = sio.loadmat(name)
            date_in_this_file = [i[0][0] for i in file['table'][1]]
            if name == sorted(filenames_model)[0] or date_in_this_file[0] >= 737392:
                new = [i for i in date_in_this_file if i not in already_date][::-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            else:
                new = [i for i in date_in_this_file if i not in already_date][::-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            old_length = len(already_date)
            already_date += new
            new_length = len(already_date)
            for i in new_index:
                already_rank.append([j[0] for j in file['table'][0][i]])
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        full_cont_before[model] = cont
    else:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model):
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            file = sio.loadmat(name)
            date_in_this_file = [i[0][0] for i in file['table'][1]]
            new = [i for i in date_in_this_file if i not in already_date][::-1]
            new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            old_length = len(already_date)
            already_date += new
            new_length = len(already_date)
            for i in new_index:
                already_rank.append([j[0] for j in file['table'][0][i]])
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        full_cont_before[model] = cont

In [5]:
#fill E34, I45
full_cont['E3'] = pd.concat([full_cont['E1'][~full_cont['E1'].index.isin(full_cont['E3'].index)],full_cont['E3']])
full_cont['E4'] = pd.concat([full_cont['E2'][~full_cont['E2'].index.isin(full_cont['E4'].index)],full_cont['E4']])
full_cont['I4'] = pd.concat([full_cont['I2'][~full_cont['I2'].index.isin(full_cont['I4'].index)],full_cont['I4']])
full_cont['I5'] = pd.concat([full_cont['I3'][~full_cont['I3'].index.isin(full_cont['I5'].index)],full_cont['I5']])
full_cont_before['E3'] = pd.concat([full_cont_before['E1'][~full_cont_before['E1'].index.isin(full_cont_before['E3'].index)],full_cont_before['E3']])
full_cont_before['E4'] = pd.concat([full_cont_before['E2'][~full_cont_before['E2'].index.isin(full_cont_before['E4'].index)],full_cont_before['E4']])
full_cont_before['I4'] = pd.concat([full_cont_before['I2'][~full_cont_before['I2'].index.isin(full_cont_before['I4'].index)],full_cont_before['I4']])
full_cont_before['I5'] = pd.concat([full_cont_before['I3'][~full_cont_before['I3'].index.isin(full_cont_before['I5'].index)],full_cont_before['I5']])

In [6]:
#data
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
industry_full = pd.read_excel('sw_industry_full.xlsx',index_col = 0)
industry_list = sorted(industry_full.iloc[0,:].dropna().unique().tolist())
mkt_cap_ard = pd.read_csv('mkt_cap_ard.csv',index_col=0)
mkt_cap_ard.columns = [int(i) for i in mkt_cap_ard.columns]
mkt_cap_ard.fillna(method='ffill',inplace=True)
amt = pd.read_csv('amt.csv',index_col=0)
amt.columns = [int(i) for i in amt.columns]
close = pd.read_csv('close.csv',index_col=0)
close.columns = [int(i) for i in close.columns]
prest_kick = pd.read_csv('prest_kick.csv',index_col = 0)
prest_kick.columns = [int(i) for i in prest_kick.columns]

In [7]:
full_cont['J2'] = full_cont['J2'].reset_index().drop_duplicates(subset = [0]).set_index(0)[1]
full_cont_before['J2'] = full_cont_before['J2'].reset_index().drop_duplicates(subset = [0]).set_index(0)[1]

In [8]:
#展开为回测参数说明
'''
通用的回测参数说明

limitation_list: 流动性限制，无默认，list中的每一项应为一个含三元素的tuple
                 (数字限制,文字限制(仅为输出文件名相关，含'_'为输出美观),百分比限制(和数字限制只会有一个生效，数字限制优先，若使用百分比限制则数字限制必须为0))
                 如(50000000,'5000万_',1)，昨日成交额大于50000000的股票会保留进行排序
                 (0,'',1)，全部股票会保留进行排序
                 (0,'70p',0.7)，昨日成交额前70%的股票会保留进行排序
buy_num_list: 购买支数，无默认，list中的每一项应为一个int，或为一个含二元素的tuple
              如[10,20,(30,50)]，最终的选股分别会为前10支，前20支，前30~50支（即不包含最头部的29支）
key_list: 测试的模型，默认为全部模型（full_cont中的全部key），自行设置时格式应类似为['A3','E1']
          新模型测试时（即最后两个名称开头为new的function）,无默认，list中的每一项应为一个含二元素的tuple
          (mat文件的名称(不含'.mat'),持仓天数)
          如('79.2', 9)，即对‘79.2.mat’进行测试，该模型持仓天数为9天
sell_list: 止盈线，默认为7%，即为最后一持仓天中若价格超过7%则卖出，list中每一项应为一个含二元素的tuple
           (数字限制，文字限制)
           如(0.07,'7_')，最后一持仓天中若价格超过7%则以max(7%,开盘涨幅)计算收益率
upper: 默认为8%，买入日开盘涨幅若大于此数值则记为未买入（若希望去除此限制，设置upper为11%（大于涨停幅度）即可）
lower: 默认为-9.9%，买入日开盘涨幅若小于此数值则记为未买入（若希望去除此限制，设置lower为-11%（小于跌停幅度）即可）

quick_test: 是否进行快速回测，默认为是
            设置为是时，最后一天的preclose-to-stop_profit_close（昨收至进行止盈了的收盘收益）即为最后一天的收益
            设置为否时，最后一天一字跌停、停牌无法卖出的股票会一直保留直到不为一字跌停且不为停牌为止，由于极端情况下停牌时间、连续跌停日会极长，计算时间可能较慢
            若不使用快速回测，生成的文件文件名在流动性限制的文字性限制之后会加入一个'_'进行区分
use_previous: 是否使用以前的回测文件，默认为是
              设置为是时，该回测的前n-2h-1天的回测数值会自动使用已有的同名文件内的内容，n为全部回测天数，h为该模型的持仓天数，只重新计算后2h+1天
              设置为否时，该回测会从头(full_cont中有记录的全部日期)进行回测
add_unfinished: 是否在后续加入前一日因开盘涨幅大于upper或小于lower或停牌而未买入的股票，默认为是
                设置为是时，第一日开盘涨幅大于upper或小于lower或停牌的股票在持仓周期（除最后一天）内若同时满足开盘涨幅小于upper且大于lower且不停牌，
                则从该天计算收益
                设置为否时，第一日开盘涨幅大于upper或小于lower或停牌的股票整个持仓周期内的收益都会被设置为0

目前理论上最准确的回测方式为设置quick_test, use_previous为False，设置add_unfinished为True
'''
print()

#backtesting functions

In [9]:
def prest_kick_normal_backtest(\
    limitation_list, buy_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    st_ori_change = pd.DataFrame()
                    kick_record = pd.DataFrame()
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont[key].xs(date_value)[:]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        non_kick_ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                        
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                            
                        if type(buy_num) == int:
                            non_kick_portfolio = non_kick_ranked_stock_universe[:buy_num]
                        else:
                            non_kick_portfolio = non_kick_ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                            
                        kick_record_add = pd.DataFrame([[sts for sts in non_kick_portfolio if sts not in portfolio]], index = [int_date_value])
                        kick_record = pd.concat([kick_record,kick_record_add])
                            
                        future_ST = sorted(list(set((STcode_by_date[STcode_by_date.index>int_date_value]\
                                              .iloc[:n_hold-1].stack().dropna().astype(int).tolist()))))
                        st_ori_change_add = pd.DataFrame([[sts for sts in portfolio if sts in future_ST]], index = [int_date_value])
                        st_ori_change = pd.concat([st_ori_change,st_ori_change_add])
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                        st_ori_change.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_st池.csv')
                        kick_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_kick池.csv')
                    else:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv')
                        st_ori_change.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_st池.csv')
                        kick_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_kick池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [10]:
def prest_kick_mktind_backtest(\
    limitation_list, ind_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider, multiplier in modify_list:
                    for key in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        n_hold = preset_hold_days(key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv'), \
                                        index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv'), \
                                        index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                             if full_cont_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            date_list = full_cont[key].index.tolist()
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        st_ori_change = pd.DataFrame()
                        kick_record = pd.DataFrame()
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            if key[0] in 'I' and int_date_value < 20190513:
                                ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                            else:
                                ranked_stock_universe = full_cont[key].xs(date_value)[:]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            non_kick_portfolio = []
                            if int_date_value < 20190225:
                                selected_industry = ranked_with_industry.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]
                            else:
                                selected_industry = ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')\
                                    *multiplier + \
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                    ].index.tolist()].sort_values().index.tolist()[:first_n]
                            for ind in selected_industry:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            for ind in selected_industry:
                                non_kick_portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt))].iloc[:ind_num].index.tolist()
                            kick_record_add = pd.DataFrame([[sts for sts in non_kick_portfolio if sts not in portfolio]], index = [int_date_value])
                            kick_record = pd.concat([kick_record,kick_record_add])
                            future_ST = list(set((STcode_by_date[STcode_by_date.index>int_date_value]\
                                              .iloc[:n_hold-1].stack().dropna().astype(int).tolist())))
                            st_ori_change_add = pd.DataFrame([[sts for sts in portfolio if sts in future_ST]], \
                                                             index = [int_date_value])
                            st_ori_change = pd.concat([st_ori_change,st_ori_change_add])
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头', key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                        kick_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_kick股票池.csv')
                        st_ori_change.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_st股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [11]:
#只在 keep 里进行选股
def keep_normal_backtest(\
    limitation_list, buy_num_list, keep = '银河证券', key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
    keep_stocks = pd.read_excel(keep+'.xlsx')['证券代码'].tolist()
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel(keep+'回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_'+keep+'股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel(keep+'回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_'+keep+'股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont[key].xs(date_value)[:]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick)) & \
                                        (pd.Series(ranked_stock_universe).isin(keep_stocks))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel(keep+'回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_'+keep+'股票池.csv')
                    else:
                        result.to_excel(keep+'回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_'+keep+'股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [12]:
#不在 kick 里进行选股
def kick_normal_backtest(\
    limitation_list, buy_num_list, kick = 'gongyedama', key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
    kick_stocks = pd.read_excel(kick+'.xlsx')['代码'].tolist()
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel(kick+'剔除回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_'+kick+'剔除股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel(kick+'剔除回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_'+kick+'剔除股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont[key].xs(date_value)[:]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick)) & \
                                        (~pd.Series(ranked_stock_universe).isin(kick_stocks))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel(kick+'剔除回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_'+kick+'剔除股票池.csv')
                    else:
                        result.to_excel(kick+'剔除回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])\
                                                +'-'+str(buy_num[1])+'_支_'+kick+'剔除股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [13]:
#不在 kick 里进行选股，使用最旧版模型
def old_kick_normal_backtest(\
    limitation_list, buy_num_list, kick = 'gongyedama', key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
    kick_stocks = pd.read_excel(kick+'.xlsx')['代码'].tolist()
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel('kick回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_kick最旧股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel('kick回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_kick最旧股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont_before[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont_before[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick)) & \
                                        (~pd.Series(ranked_stock_universe).isin(kick_stocks))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel('kick回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.\
                        to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_kick最旧股票池.csv')
                    else:
                        result.to_excel('kick回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.\
                        to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_kick最旧股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [14]:
#正常的选股
def normal_backtest(\
    limitation_list, buy_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont[key].xs(date_value)[:]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                    else:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [15]:
#市值行业
def mktind_backtest(\
    limitation_list, ind_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider, multiplier in modify_list:
                    for key in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        n_hold = preset_hold_days(key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv'), \
                                        index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv'), \
                                        index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                             if full_cont_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            date_list = full_cont[key].index.tolist()
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            if key[0] in 'I' and int_date_value < 20190513:
                                ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                            else:
                                ranked_stock_universe = full_cont[key].xs(date_value)[:]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            if int_date_value < 20190225:
                                selected_industry = ranked_with_industry.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]
                            else:
                                selected_industry = ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')\
                                    *multiplier + \
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                    ].index.tolist()].sort_values().index.tolist()[:first_n]
                            for ind in selected_industry:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头', key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [16]:
#魔改市值行业
def mktind_m_backtest(\
    limitation_list, ind_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider,multiplier in modify_list:
                    for key in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        n_hold = preset_hold_days(key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv'), \
                                index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv'), \
                                index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                             if full_cont_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            date_list = full_cont[key].index.tolist()
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            if key[0] in 'I' and int_date_value < 20190513:
                                ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                            else:
                                ranked_stock_universe = full_cont[key].xs(date_value)[:]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            for ind in ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头',key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [17]:
#正常的选股，使用最旧版模型
def old_normal_backtest(\
    limitation_list, buy_num_list, key_list = full_cont_before.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_最旧股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_最旧股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        #这一天前一天是哪天？
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧股票池.csv')
                    else:
                        result.to_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_最旧股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [18]:
#市值行业，使用最旧版模型
def old_mktind_backtest(\
    limitation_list, ind_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider, multiplier in modify_list:
                    for key in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        n_hold = preset_hold_days(key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧股票池.csv'), \
                                        index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧权重.csv'), \
                                        index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [full_cont_date for full_cont_date in full_cont_before[key].index.tolist() \
                                             if full_cont_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            date_list = full_cont_before[key].index.tolist()
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            if int_date_value < 20190225:
                                selected_industry = ranked_with_industry.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]
                            else:
                                selected_industry = ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')\
                                    *multiplier + \
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                    ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                    ].index.tolist()].sort_values().index.tolist()[:first_n]
                            for ind in selected_industry:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头', key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+\
                                        '市值行业_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [19]:
#魔改市值行业，使用最旧版模型
def old_mktind_m_backtest(\
    limitation_list, ind_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider,multiplier in modify_list:
                    for key in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        n_hold = preset_hold_days(key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧股票池.csv'), \
                                index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧权重.csv'), \
                                index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                             if full_cont_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            date_list = full_cont[key].index.tolist()
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            for ind in ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头',key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测最旧_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_最旧权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [20]:
#沪深300拟合，基本不用，不太好
def hs300_backtest(\
    limitation_list, buy_num_list, key_list = full_cont.keys(), sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key in key_list:
                    start_timer = datetime.now()
                    print('处理',key)
                    n_hold = preset_hold_days(key)
                    cost = total_cost/n_hold
                    try:
                        previous_flag = 1
                        previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_HS300增强_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                        previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'HS300增强_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv'), \
                            index_col = 0)
                        previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'HS300增强_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv'), \
                            index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [full_cont_date for full_cont_date in full_cont[key].index.tolist() \
                                         if full_cont_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                            previous_weight.columns = [int(i) for i in previous_weight.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        date_list = full_cont[key].index.tolist()
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = hsIcData.loc[date_list,:][5]/hsIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (hsIcData.loc[date_list,:][5]/hsIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    #记录收益、持仓的list
                    full_portfolio = []
                    full_portfolio_ret = []
                    full_portfolio_weight = []
                    #记录日期的list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in hsIcData.index:
                            continue
                        print('处理第',date_index+1,'天')
                        #日期转换
                        int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                            industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                            industry.columns = ['ind']
                        else:
                            industry = industry_full.loc[[20190211],:].T
                            industry.columns = ['ind']
                        #全部（排名）
                        if key[0] in 'I' and int_date_value < 20190513:
                            ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                        else:
                            ranked_stock_universe = full_cont[key].xs(date_value)[:]
                        #这一天前一天是哪天？
                        today_hs300_index = hsIcData.index.tolist().index(date_value)
                        keep_days_list = hsIcData.index.tolist()[today_hs300_index:today_hs300_index + n_hold]
                        #前一天成交额（按绝对值）
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        #ST股票
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #先踢掉ST、成交额不满足条件的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                        origin = ranked_stock_universe[:buy_num]
                        #HS300行业权重
                        members = hs300members.xs(int_date_value).tolist()
                        mid_members_industry = (pd.concat([(\
                            adjusted_weight_coef.shift(1).xs(int_date_value).loc[members]/\
                            adjusted_weight_coef.shift(1).xs(int_date_value).loc[members]\
                            .sum()).sort_values(),industry['ind'].loc[members]],axis=1).groupby('ind').sum().iloc[:,0]*100)\
                            .sort_values(ascending=False)
                        #ranked_stock_universe个股行业
                        ranked_with_industry = industry.loc[ranked_stock_universe].dropna()
                        ranked_with_industry['rrank'] = range(len(ranked_with_industry))
                        exp = ranked_with_industry.sort_values('rrank').dropna()
                        exp['rank'] = range(1,len(exp)+1)
                        adj_list = []
                        name_list = []
                        for included_ind in exp['ind'].unique():
                            sub = exp[exp['ind'] == included_ind]
                            adjust = (exp['rank'].mean() - sub['rank'].mean()) * len(sub)
                            adj_list.append(adjust)
                            name_list.append(included_ind)
                        adj_list = [i/np.max(adj_list) * 20 for i in adj_list]
                        members_industry = pd.DataFrame(list(zip(name_list,adj_list))).set_index(0)[1] + mid_members_industry
                        members_industry = members_industry[members_industry>0]
                        members_industry = members_industry/members_industry.sum()
                        members_industry = round(members_industry.sort_values(ascending=False)*buy_num)
                        res = pd.concat([industry.loc[origin]['ind'].value_counts()/buy_num * 100,\
                                   members_industry/members_industry.sum() * 100,mid_members_industry],axis=1)
                        res.columns = ['origin','adjusted','hs300']
                        res.fillna(0,inplace=True)
                        res['result'] = (res['origin'] - res['adjusted']) * 1/2 + res['adjusted']
                        members_industry = round(res['result']/res['result'].sum() * buy_num)
                        portfolio = []
                        for indname in members_industry.index:
                            add = ranked_with_industry[(ranked_with_industry['ind'] == indname) & \
                                                       (~ranked_with_industry.index.isin(today_prest_kick))].index.tolist()[:int(members_industry.xs(indname))]
                            portfolio += add
                        p_data = adjusted_weight_coef.shift(1).xs(int_date_value).loc[portfolio]
                        portfolio_weight = p_data/(p_data.sum())
                        print(portfolio_weight.max()/portfolio_weight.min())
                        portfolio_weight = (portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2).sort_values()
                        print(portfolio_weight.max()/portfolio_weight.min())
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        def process_weight(df):
                            return df * portfolio_weight
                        portfolio_ret = portfolio_ret.apply(process_weight)
                        full_portfolio_ret.append(portfolio_ret)
                        full_portfolio_weight.append(portfolio_weight)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.sum() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                        portfolio_weight_record = pd.concat([previous_weight, \
                            portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                    result.to_excel('回测_'+str(n_hold)+'_('+key+')_HS300增强_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                    portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'HS300增强_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                    portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'HS300增强_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv')
                    #计算时间
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()

In [21]:
#单独给一个mat文件进行新模型实验使用，正常的选股
def new_normal_backtest(\
    limitation_list, buy_num_list, key_list, sell_list = [(0.07, '7_')], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for limitation,lim_str,lim_pct in limitation_list:
            if not quick_test:
                lim_str += '_'
            for buy_num in buy_num_list:
                for key,n_hold in key_list:
                    start_timer = datetime.now()
                    full_portfolio = []
                    print('处理',key)
                    cost = total_cost/n_hold
                    print(cost)
                    try:
                        previous_flag = 1
                        if type(buy_num) == int:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num)+'_支_股票池.csv'), \
                                index_col = 0)
                        else:
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+lim_str+\
                                sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv'), \
                                index_col = 0)
                        previous_date = previous.index.tolist()[-2*n_hold]
                    except:
                        previous_flag = 0
                        previous = None
                    if use_previous == False:
                        previous_flag = 0
                        previous = None
                    if previous_flag == 1:
                        os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
                        file = sio.loadmat(key+'.mat')
                        os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
                        if len(previous) >= 2*n_hold:
                            ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                            previous_date%10000//100,\
                                                                            previous_date%100))+366
                            date_list = [table_date for table_date in [i[0][0] for i in file['table'][1]] \
                                         if table_date > ordinal_previous_date]
                            previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                            previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                            previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                    else:
                        os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
                        file = sio.loadmat(key+'.mat')
                        os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
                        date_list = [i[0][0] for i in file['table'][1]]
                    index_ret = pd.DataFrame()
                    for n in range(n_hold):
                        if n == 0:
                            index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                        else:
                            index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                        index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                    full_portfolio_ret = []
                    #日期list
                    dates = []
                    for date_index,date_value in enumerate(date_list):
                        if date_value not in zzIcData.index:
                            continue
                        int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                        if int_date_value in prest_kick.index.tolist():
                            today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                        else:
                            today_prest_kick = []
                        dates.append(int_date_value)
                        today_zz500_index = zzIcData.index.tolist().index(date_value)
                        keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                        print('处理第',date_index+1,'天')
                        #全部（排名）
                        if previous_flag:
                            ranked_stock_universe = [int(i[0]) for i in file['table'][0][len(previous)-n_hold+date_index] if pd.notnull(i)]
                        else:
                            ranked_stock_universe = [int(i[0]) for i in file['table'][0][date_index] if pd.notnull(i)]
                        #前一天成交额
                        if limitation != 0:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                        elif limitation == 0 and lim_pct != 1:
                            yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                            yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                        else:
                            yester_amt = []
                        STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                        #所有的需要踢掉的
                        ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                        (~pd.Series(ranked_stock_universe).isin(today_prest_kick))].tolist()
                        #计算整体收益
                        if type(buy_num) == int:
                            portfolio = ranked_stock_universe[:buy_num]
                        else:
                            portfolio = ranked_stock_universe[(buy_num[0]-1):(buy_num[1]-1)]
                        portfolio_ret = pd.DataFrame()
                        for day_index in range(len(keep_days_list)):
                            if day_index == 0:
                                portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            elif day_index == n_hold-1:
                                portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            else:
                                portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                        sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                        sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                        special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                        portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                        if not quick_test:
                            last_day = keep_days_list[-1]
                            while(True):
                                try:
                                    next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                except:
                                    break
                                if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                    (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                        ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio])/\
                                        (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                        is_trade.xs(last_day,level=1).loc[portfolio]))\
                                        .fillna(0)],axis=1)
                                    last_day = next_day
                                else:
                                    break
                        if add_unfinished:
                            for sp in special:
                                for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                    try:
                                        future_co = co.xs((sp,keep_day))
                                        future_amount = amount.xs((sp,keep_day))
                                    except:
                                        portfolio_ret.loc[sp,:] = 0
                                        break
                                    if lower < future_co < upper and future_amount > 0:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                        break
                                    else:
                                        portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                        if keep_day_index + 2 == len(keep_days_list):
                                            portfolio_ret.loc[sp,:] = 0
                        else:
                            portfolio_ret.loc[special,:] = 0
                        full_portfolio.append(portfolio)
                        full_portfolio_ret.append(portfolio_ret)
                    #阶梯均值dataframe
                    full_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.mean()).T])
                    #阶梯均值-中证500dataframe
                    full_exc_mean_portfolio_ret = pd.DataFrame()
                    for index,value in enumerate(full_portfolio_ret):
                        df = value.copy()
                        df.columns = range(index,index+len(list(df)))
                        index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                        if len(list(df)) > n_hold:
                            for index_,value_ in enumerate(list(df)):
                                if index_>=n_hold:
                                    index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                        (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                        index_ret_.index = df.mean().index
                        exc = df.mean() - index_ret_
                        full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                    #多头和对冲的计算
                    duotou = ((full_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    duichong = ((full_exc_mean_portfolio_ret.sum()/n_hold)-cost).cumsum()
                    #保存到excel
                    result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                    result.index = dates
                    result.columns = [key+' 多头',key+' 对冲']
                    portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                    if previous_flag == 1:
                        result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                        portfolio_record = pd.concat([previous_portfolio, \
                            portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                    if type(buy_num) == int:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                    else:
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+lim_str+sell_str+str(buy_num[0])+'-'+str(buy_num[1])+'_支_股票池.csv')
                    print((datetime.now() - start_timer))
                    start_timer = datetime.now()
    return full_portfolio_ret

In [22]:
#单独给一个mat文件进行新模型实验使用，魔改市值行业
def new_mktind_m_backtest(\
    limitation_list, ind_num_list, key_list, sell_list = [(0.07, '7_')], modify_list = [(2,0)], total_cost = 0.005, \
    quick_test = True, use_previous = True, add_unfinished = True, upper = 0.08, lower = -0.11):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for sell,sell_str in sell_list:
        cc_l = cc.copy()
        cc_l[ch>sell] = sell
        cc_l[(co>sell)&(ch>sell)] = co[(co>sell)&(ch>sell)]
        for first_n,ind_num in ind_num_list:
            for limitation,lim_str,lim_pct in limitation_list:
                if not quick_test:
                    lim_str += '_'
                for divider,multiplier in modify_list:
                    for key,n_hold in key_list:
                        buy_num = first_n * ind_num
                        start_timer = datetime.now()
                        full_portfolio = []
                        full_portfolio_weight = []
                        print('处理',key)
                        cost = total_cost/n_hold
                        try:
                            previous_flag = 1
                            previous = pd.read_excel('回测_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx', index_col = 0)
                            previous_portfolio = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv'), \
                                index_col = 0)
                            previous_weight = pd.read_csv(open(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                                str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv'), \
                                index_col = 0)
                            previous_date = previous.index.tolist()[-2*n_hold]
                        except:
                            previous_flag = 0
                            previous = None
                        if use_previous == False:
                            previous_flag = 0
                            previous = None
                        if previous_flag == 1:
                            os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
                            file = sio.loadmat(key+'.mat')
                            os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
                            if len(previous) >= 2*n_hold:
                                ordinal_previous_date = date.toordinal(datetime(previous_date//10000,\
                                                                                previous_date%10000//100,\
                                                                                previous_date%100))+366
                                date_list = [table_date for table_date in [i[0][0] for i in file['table'][1]] \
                                             if table_date > ordinal_previous_date]
                                previous_portfolio = previous_portfolio[previous_portfolio.index <= previous.index.tolist()[-n_hold]]
                                previous_portfolio.columns = [int(i) for i in previous_portfolio.columns]
                                previous_weight = previous_weight[previous_weight.index <= previous.index.tolist()[-n_hold]]
                                previous_weight.columns = [int(i) for i in previous_weight.columns]
                                previous = previous[previous.index <= previous.index.tolist()[-n_hold]]
                        else:
                            os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
                            file = sio.loadmat(key+'.mat')
                            os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
                            date_list = [i[0][0] for i in file['table'][1]]
                        index_ret = pd.DataFrame()
                        for n in range(n_hold):
                            if n == 0:
                                index_ret_add = zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][2]-1
                            else:
                                index_ret_add = (zzIcData.loc[date_list,:][5]/zzIcData.loc[date_list,:][5].shift(1)-1).shift(-n).fillna(0)
                            index_ret = pd.concat([index_ret,index_ret_add],axis=1)
                        full_portfolio_ret = []
                        #日期list
                        dates = []
                        for date_index,date_value in enumerate(date_list):
                            if date_value not in zzIcData.index:
                                continue
                            #日期转换
                            int_date_value = int(str(date.fromordinal(date_value-366)).replace('-',''))
                            if int_date_value in prest_kick.index.tolist():
                                today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                            else:
                                today_prest_kick = []
                            dates.append(int_date_value)
                            if int(get_n_week_date_before(str(int_date_value))) in industry_full.index.tolist():
                                industry = industry_full.loc[[int(get_n_week_date_before(str(int_date_value)))],:].T
                                industry.columns = ['ind']
                            else:
                                industry = industry_full.loc[[20190211],:].T
                                industry.columns = ['ind']
                            print('处理第',date_index+1,'天')
                            #全部（排名）
                            if previous_flag:
                                ranked_stock_universe = [int(i[0]) for i in file['table'][0][len(previous)-n_hold+date_index] if pd.notnull(i)]
                            else:
                                ranked_stock_universe = [int(i[0]) for i in file['table'][0][date_index] if pd.notnull(i)]
                            #这一天前一天是哪天？
                            today_zz500_index = zzIcData.index.tolist().index(date_value)
                            keep_days_list = zzIcData.index.tolist()[today_zz500_index:today_zz500_index + n_hold]
                            #前一天成交额
                            if limitation != 0:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                            elif limitation == 0 and lim_pct != 1:
                                yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                                yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                            else:
                                yester_amt = []
                            STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                            #所有的需要踢掉的
                            ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                        (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                        (~pd.Series(ranked_stock_universe).isin(yester_amt))].tolist()
                            yester_mkt_cap = mkt_cap_ard.shift(1).xs(int_date_value).loc[ranked_stock_universe]
                            def aa(df):
                                def standardize(ts):
                                    ts = ts.copy().dropna()
                                    return (ts - ts.mean()) / ts.std()
                                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
                            ranked_with_industry = industry.loc[ranked_stock_universe]
                            standardized_yester_mkt_cap = pd.concat([ranked_with_industry,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
                            standardized_yester_mkt_cap.fillna(0,inplace=True)
                            ranked_with_industry['rank'] = range(len(ranked_with_industry))
                            portfolio = []
                            for ind in ((ranked_with_industry.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank'][\
                                ranked_with_industry.iloc[:int(len(ranked_with_industry)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                                portfolio += ranked_with_industry[(ranked_with_industry['ind'] == ind) & \
                                    (~ranked_with_industry.index.isin(yester_amt)) & \
                                    (~ranked_with_industry.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
                            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
                            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                                loc[portfolio].iloc[:,1])
                            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
                            while(True):
                                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                                    break
                                else:
                                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
                            portfolio_ret = pd.DataFrame()
                            for day_index in range(len(keep_days_list)):
                                if day_index == 0:
                                    portfolio_ret = pd.concat([portfolio_ret,oc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                elif day_index == n_hold-1:
                                    portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                                else:
                                    portfolio_ret = pd.concat([portfolio_ret,cc.xs(keep_days_list[day_index],level=4).loc[portfolio]],axis=1)
                            sub_co = co.xs(keep_days_list[0],level=4).loc[portfolio]
                            sub_amount = amount.xs(keep_days_list[0],level=4).loc[portfolio]
                            special = sub_co[(sub_co>=upper)|(sub_co<=lower)].index.tolist() + sub_amount[sub_amount==0].index.tolist()
                            portfolio = [sst for sst in portfolio_ret.index if sst not in special]
                            if not quick_test:
                                last_day = keep_days_list[-1]
                                while(True):
                                    try:
                                        next_day = next(dt for dt in zzIcData.index.tolist() if dt > last_day)
                                    except:
                                        break
                                    if (is_dieting.xs(last_day,level=1).loc[portfolio].sum()) or \
                                        (is_trade.xs(last_day,level=1).loc[portfolio].sum()):
                                        portfolio_ret = pd.concat([portfolio_ret,cc_l.xs(next_day,level=1).loc[portfolio] * \
                                            ((is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio])/\
                                            (is_dieting.xs(last_day,level=1).loc[portfolio] + \
                                            is_trade.xs(last_day,level=1).loc[portfolio]))\
                                            .fillna(0)],axis=1)
                                        last_day = next_day
                                    else:
                                        break
                            if add_unfinished:
                                for sp in special:
                                    for keep_day_index, keep_day in enumerate(keep_days_list[:-1]):
                                        try:
                                            future_co = co.xs((sp,keep_day))
                                            future_amount = amount.xs((sp,keep_day))
                                        except:
                                            portfolio_ret.loc[sp,:] = 0
                                            break
                                        if lower < future_co < upper and future_amount > 0:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = future_co
                                            break
                                        else:
                                            portfolio_ret.iloc[portfolio_ret.index.tolist().index(sp),keep_day_index] = 0
                                            if keep_day_index + 2 == len(keep_days_list):
                                                portfolio_ret.loc[sp,:] = 0
                            else:
                                portfolio_ret.loc[special,:] = 0
                            full_portfolio.append(portfolio)
                            def process_weight(df):
                                return df * portfolio_weight
                            portfolio_ret = portfolio_ret.apply(process_weight)
                            full_portfolio_ret.append(portfolio_ret)
                            full_portfolio_weight.append(portfolio_weight)
                        #阶梯均值dataframe
                        full_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            full_mean_portfolio_ret = pd.concat([full_mean_portfolio_ret,pd.DataFrame(df.sum()).T])
                        #阶梯均值-中证500dataframe
                        full_exc_mean_portfolio_ret = pd.DataFrame()
                        for index,value in enumerate(full_portfolio_ret):
                            df = value.copy()
                            df.columns = range(index,index+len(list(df)))
                            index_ret_ = index_ret.iloc[index,:].iloc[:len(list(df))]
                            if len(list(df)) > n_hold:
                                for index_,value_ in enumerate(list(df)):
                                    if index_>=n_hold:
                                        index_ret_ = index_ret_.append(pd.Series([index_ret.iloc[index+index_-n_hold+1,-1] *\
                                            (len(df.iloc[:,index_][df.iloc[:,index_]!=0])/len(df))],index=[0]))
                            index_ret_.index = df.mean().index
                            exc = df.sum() - index_ret_
                            full_exc_mean_portfolio_ret = pd.concat([full_exc_mean_portfolio_ret,pd.DataFrame(exc).T])
                        #多头和对冲的计算
                        duotou = ((full_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        duichong = ((full_exc_mean_portfolio_ret.sum() / n_hold)-cost).cumsum()
                        #保存到excel
                        result = pd.concat([duotou,duichong],axis=1).iloc[:len(dates),:]
                        result.index = dates
                        result.columns = [key+' 多头',key+' 对冲']
                        portfolio_record = pd.DataFrame(full_portfolio,index=dates)
                        portfolio_weight_record = pd.DataFrame(full_portfolio_weight,index=dates)
                        if previous_flag == 1:
                            result = pd.concat([previous,result.diff()[~result.diff().index.isin(previous.index)].cumsum()+previous.iloc[-1]])
                            portfolio_record = pd.concat([previous_portfolio, \
                                portfolio_record[~portfolio_record.index.isin(previous_portfolio.index)]])
                            portfolio_weight_record = pd.concat([previous_weight, \
                                portfolio_weight_record[~portfolio_weight_record.index.isin(previous_weight.index)]])
                        result.to_excel('回测_'+str(n_hold)+'_('+key+')_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'支.xlsx')
                        portfolio_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_股票池.csv')
                        portfolio_weight_record.to_csv(str(n_hold)+'_首组_'+key+'_'+'市值行业_m_'+str(divider)+'_'+\
                            str(multiplier)+'_'+str(ind_num)+'_'+str(first_n)+'_'+lim_str+sell_str+str(buy_num)+'_支_权重.csv')
                        print((datetime.now() - start_timer))
                        start_timer = datetime.now()

In [8]:
def normal_stock_ranking(limitation_list, key_list = full_cont.keys()):
    os.chdir(r'F:\金湖回测系统\6日循环\结果存放文件夹')
    for limitation,lim_str,lim_pct in limitation_list:
        for key in key_list:
            start_timer = datetime.now()
            full_portfolio = []
            print('处理',key)
            date_list = full_cont[key].index.tolist()
            dates = []
            for date_index,date_value in enumerate(date_list):
                if date_value not in zzIcData.index:
                    continue
                int_date_value =int(str(date.fromordinal(date_value-366)).replace('-',''))
                if int_date_value in prest_kick.index.tolist():
                    today_prest_kick = prest_kick.xs(int_date_value).dropna().tolist()
                else:
                    today_prest_kick = []
                dates.append(int_date_value)
                print('处理第',date_index+1,'天')
                #全部（排名）
                if key in ['I1','I2','I3'] and int_date_value < 20190513:
                    ranked_stock_universe = full_cont_before[key].xs(date_value)[:]
                else:
                    ranked_stock_universe = full_cont[key].xs(date_value)[:]
                #前一天成交额
                if limitation != 0:
                    yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                    yester_amt = yester_amt[~(yester_amt>limitation)].index.tolist()
                elif limitation == 0 and lim_pct != 1:
                    yester_amt = shifted_amount.xs(date_value,level=4).fillna(0)
                    yester_amt = yester_amt.sort_values().iloc[:int((1-lim_pct)*len(yester_amt))].index.tolist()
                else:
                    yester_amt = []
                STcode = [int(stcode) for stcode in STcode_by_date.xs(int_date_value).dropna().tolist()]
                #所有的需要踢掉的
                ranked_stock_universe = pd.Series(ranked_stock_universe)[\
                                (~pd.Series(ranked_stock_universe).isin(STcode)) & \
                                (~pd.Series(ranked_stock_universe).isin(yester_amt)) & \
                                (~pd.Series(ranked_stock_universe).isin(today_prest_kick))].tolist()
                portfolio = ranked_stock_universe[:]
                full_portfolio.append(portfolio)
            portfolio_record = pd.DataFrame(full_portfolio,index=dates)
            portfolio_record.to_csv(key+'_'+lim_str+'_股票池.csv')
            print((datetime.now() - start_timer))
            start_timer = datetime.now()

In [9]:
normal_stock_ranking(limitation_list=[(50000000,'5000万_',1),(0,'',1)])

处理 A2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天
处理第 62 天
处理第 63 天
处理第 64 天
处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理第 85 天
处理第 86 天
处理第 87 天
处理第 88 天
处理第 89 天
处理第 90 天
处理第 91 天
处理第 92 天
处理第 93 天
处理第 94 天
处理第 95 天
处理第 96 天
处理第 97 天
处理第 98 天
处理第 99 天
处理第 100 天
处理第 101 天
处理第 102 天
处理第 103 天
处理第 104 天
处理第 105 天
处理第 106 天
处理第 107 天
处理第 108 天
处理第 109 天
处理第 110 天
处理

处理第 392 天
处理第 393 天
处理第 394 天
处理第 395 天
处理第 396 天
处理第 397 天
处理第 398 天
处理第 399 天
处理第 400 天
处理第 401 天
处理第 402 天
处理第 403 天
处理第 404 天
处理第 405 天
处理第 406 天
处理第 407 天
处理第 408 天
处理第 409 天
处理第 410 天
处理第 411 天
处理第 412 天
处理第 413 天
处理第 414 天
处理第 415 天
处理第 416 天
处理第 417 天
处理第 418 天
处理第 419 天
处理第 420 天
处理第 421 天
处理第 422 天
处理第 423 天
处理第 424 天
处理第 425 天
处理第 426 天
处理第 427 天
处理第 428 天
处理第 429 天
处理第 430 天
处理第 431 天
处理第 432 天
处理第 433 天
处理第 434 天
处理第 435 天
0:00:10.358010
处理 A4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天

处理第 410 天
0:00:09.837532
处理 B3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天
处理第 62 天
处理第 63 天
处理第 64 天
处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理第 85 天
处理第 86 天
处理第 87 天
处理第 88 天
处理第 89 天
处理第 90 天
处理第 91 天
处理第 92 天
处理第 93 天
处理第 94 天
处理第 95 天
处理第 96 天
处理第 97 天
处理第 98 天
处理第 99 天
处理第 100 天
处理第 101 天
处理第 102 天
处理第 103 天
处理第 104 天
处理第 105 天
处理第 106 天
处理第 107 天
处理第 108

处理第 126 天
处理第 127 天
处理第 128 天
处理第 129 天
处理第 130 天
处理第 131 天
处理第 132 天
处理第 133 天
处理第 134 天
处理第 135 天
处理第 136 天
处理第 137 天
处理第 138 天
处理第 139 天
处理第 140 天
处理第 141 天
处理第 142 天
处理第 143 天
处理第 144 天
处理第 145 天
处理第 146 天
处理第 147 天
处理第 148 天
处理第 149 天
处理第 150 天
处理第 151 天
处理第 152 天
处理第 153 天
处理第 154 天
处理第 155 天
处理第 156 天
处理第 157 天
处理第 158 天
处理第 159 天
处理第 160 天
处理第 161 天
处理第 162 天
处理第 163 天
处理第 164 天
处理第 165 天
处理第 166 天
处理第 167 天
处理第 168 天
处理第 169 天
处理第 170 天
处理第 171 天
处理第 172 天
处理第 173 天
处理第 174 天
处理第 175 天
处理第 176 天
处理第 177 天
处理第 178 天
处理第 179 天
处理第 180 天
处理第 181 天
处理第 182 天
处理第 183 天
处理第 184 天
处理第 185 天
处理第 186 天
处理第 187 天
处理第 188 天
处理第 189 天
处理第 190 天
处理第 191 天
处理第 192 天
处理第 193 天
处理第 194 天
处理第 195 天
处理第 196 天
处理第 197 天
处理第 198 天
处理第 199 天
处理第 200 天
处理第 201 天
处理第 202 天
处理第 203 天
处理第 204 天
处理第 205 天
处理第 206 天
处理第 207 天
处理第 208 天
处理第 209 天
处理第 210 天
处理第 211 天
处理第 212 天
处理第 213 天
处理第 214 天
处理第 215 天
处理第 216 天
处理第 217 天
处理第 218 天
处理第 219 天
处理第 220 天
处理第 221 天
处理第 222 天
处理第 223 天
处理第 224 天
处理第 225 天


处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理第 85 天
处理第 86 天
处理第 87 天
处理第 88 天
处理第 89 天
处理第 90 天
处理第 91 天
处理第 92 天
处理第 93 天
处理第 94 天
处理第 95 天
处理第 96 天
处理第 97 天
处理第 98 天
处理第 99 天
处理第 100 天
处理第 101 天
处理第 102 天
处理第 103 天
处理第 104 天
处理第 105 天
处理第 106 天
处理第 107 天
处理第 108 天
处理第 109 天
处理第 110 天
处理第 111 天
处理第 112 天
处理第 113 天
处理第 114 天
处理第 115 天
处理第 116 天
处理第 117 天
处理第 118 天
处理第 119 天
处理第 120 天
处理第 121 天
处理第 122 天
处理第 123 天
处理第 124 天
处理第 125 天
处理第 126 天
处理第 127 天
处理第 128 天
处理第 129 天
处理第 130 天
处理第 131 天
处理第 132 天
处理第 133 天
处理第 134 天
处理第 135 天
处理第 136 天
处理第 137 天
处理第 138 天
处理第 139 天
处理第 140 天
处理第 141 天
处理第 142 天
处理第 143 天
处理第 144 天
处理第 145 天
处理第 146 天
处理第 147 天
处理第 148 天
处理第 149 天
处理第 150 天
处理第 151 天
处理第 152 天
处理第 153 天
处理第 154 天
处理第 155 天
处理第 156 天
处理第 157 天
处理第 158 天
处理第 159 天
处理第 160 天
处理第 161 天
处理第 162 天
处理第 163 天
处理第 164 天
处理第 165 天
处理第 166 天
处理第 167 天
处理第 1

...

处理第 151 天
处理第 152 天
处理第 153 天
处理第 154 天
处理第 155 天
处理第 156 天
处理第 157 天
处理第 158 天
处理第 159 天
处理第 160 天
处理第 161 天
处理第 162 天
处理第 163 天
处理第 164 天
处理第 165 天
处理第 166 天
处理第 167 天
处理第 168 天
处理第 169 天
处理第 170 天
处理第 171 天
处理第 172 天
处理第 173 天
处理第 174 天
处理第 175 天
处理第 176 天
处理第 177 天
处理第 178 天
处理第 179 天
处理第 180 天
处理第 181 天
处理第 182 天
处理第 183 天
处理第 184 天
处理第 185 天
处理第 186 天
处理第 187 天
处理第 188 天
处理第 189 天
处理第 190 天
处理第 191 天
处理第 192 天
处理第 193 天
处理第 194 天
处理第 195 天
处理第 196 天
处理第 197 天
处理第 198 天
处理第 199 天
处理第 200 天
处理第 201 天
处理第 202 天
处理第 203 天
处理第 204 天
处理第 205 天
处理第 206 天
处理第 207 天
处理第 208 天
处理第 209 天
处理第 210 天
处理第 211 天
处理第 212 天
处理第 213 天
处理第 214 天
处理第 215 天
处理第 216 天
处理第 217 天
处理第 218 天
处理第 219 天
处理第 220 天
处理第 221 天
处理第 222 天
处理第 223 天
处理第 224 天
处理第 225 天
处理第 226 天
处理第 227 天
处理第 228 天
处理第 229 天
处理第 230 天
处理第 231 天
处理第 232 天
处理第 233 天
处理第 234 天
处理第 235 天
处理第 236 天
处理第 237 天
处理第 238 天
处理第 239 天
处理第 240 天
处理第 241 天
处理第 242 天
处理第 243 天
处理第 244 天
处理第 245 天
处理第 246 天
处理第 247 天
处理第 248 天
处理第 249 天
处理第 250 天


处理第 320 天
处理第 321 天
处理第 322 天
处理第 323 天
处理第 324 天
处理第 325 天
处理第 326 天
处理第 327 天
处理第 328 天
处理第 329 天
处理第 330 天
处理第 331 天
处理第 332 天
处理第 333 天
处理第 334 天
处理第 335 天
处理第 336 天
0:00:04.029470
处理 E4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天
处理第 62 天
处理第 63 天
处理第 64 天
处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理第 85 天
处理第 86 天
处理第 87 天
处理第 88 天
处理第 89 天
处理第 90 天
处理第 91 天

处理第 190 天
处理第 191 天
处理第 192 天
处理第 193 天
处理第 194 天
处理第 195 天
处理第 196 天
处理第 197 天
处理第 198 天
处理第 199 天
处理第 200 天
处理第 201 天
处理第 202 天
处理第 203 天
处理第 204 天
处理第 205 天
处理第 206 天
处理第 207 天
处理第 208 天
处理第 209 天
处理第 210 天
处理第 211 天
处理第 212 天
处理第 213 天
处理第 214 天
处理第 215 天
处理第 216 天
处理第 217 天
处理第 218 天
处理第 219 天
处理第 220 天
处理第 221 天
处理第 222 天
处理第 223 天
处理第 224 天
处理第 225 天
处理第 226 天
处理第 227 天
处理第 228 天
处理第 229 天
处理第 230 天
处理第 231 天
处理第 232 天
处理第 233 天
处理第 234 天
处理第 235 天
处理第 236 天
处理第 237 天
处理第 238 天
处理第 239 天
处理第 240 天
处理第 241 天
处理第 242 天
处理第 243 天
处理第 244 天
处理第 245 天
处理第 246 天
处理第 247 天
处理第 248 天
处理第 249 天
处理第 250 天
处理第 251 天
处理第 252 天
处理第 253 天
处理第 254 天
处理第 255 天
处理第 256 天
处理第 257 天
处理第 258 天
处理第 259 天
处理第 260 天
处理第 261 天
处理第 262 天
处理第 263 天
处理第 264 天
处理第 265 天
处理第 266 天
处理第 267 天
处理第 268 天
处理第 269 天
处理第 270 天
处理第 271 天
处理第 272 天
处理第 273 天
处理第 274 天
处理第 275 天
处理第 276 天
处理第 277 天
处理第 278 天
处理第 279 天
处理第 280 天
处理第 281 天
处理第 282 天
处理第 283 天
处理第 284 天
处理第 285 天
处理第 286 天
处理第 287 天
处理第 288 天
处理第 289 天


处理第 163 天
处理第 164 天
处理第 165 天
处理第 166 天
处理第 167 天
处理第 168 天
处理第 169 天
处理第 170 天
处理第 171 天
处理第 172 天
处理第 173 天
处理第 174 天
处理第 175 天
处理第 176 天
处理第 177 天
处理第 178 天
处理第 179 天
处理第 180 天
处理第 181 天
处理第 182 天
处理第 183 天
处理第 184 天
处理第 185 天
0:00:02.625972
处理 I2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天
处理第 62 天
处理第 63 天
处理第 64 天
处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理

0:00:00.324133
处理 K4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
0:00:00.250303
处理 K5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
0:00:00.245343
处理 K6
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
0:00:00.250333
处理 K7
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
0:00:00.269308


In [ ]:
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
prest_kick = pd.read_csv('prest_kick_empty.csv',index_col = 0)
prest_kick.columns = [int(i) for i in prest_kick.columns]

In [ ]:
new_normal_backtest(limitation_list=[(50000000,'5000万_',1),(0,'',1)], buy_num_list=[50,], \
                    key_list=[('8.31_2_handout_a2_1812_1846',2), ('8.31_2_handout_a1_1812_1840',2),], \
                    use_previous=False, lower=-0.11)

In [23]:
turn_use_previous = True

In [24]:
normal_backtest([(50000000,'5000万_',1),(0,'',1)], [50], use_previous = turn_use_previous, lower=-0.11)

处理 A2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.712419
处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.406939
处理 A4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.415890
处理 A5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.456750
处理 B1
处理第 1 天
处理第 2 天
处理第 3 天
0:00:00.405917
处理 B3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.493677
处理 B4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.432837
处理 C2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.420873
处理 C3
处理第 1 天
处理第 2 天
处理第 3 天
0:00:00.373039
处理 C4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.359002
处理 C5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.363029
处理 D1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.411898
处理 D2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.428828
处理 E1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:01.351409
处理 E2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:01.398259
处理 E3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天


In [25]:
old_normal_backtest([(50000000,'5000万_',1)], [50], use_previous = turn_use_previous, lower=-0.11)

处理 A2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.475729
处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.403907
处理 A4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.398934
处理 A5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.420874
处理 B1
处理第 1 天
处理第 2 天
处理第 3 天
0:00:00.365022
处理 B3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.431839
处理 B4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.398932
处理 C2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.407909
处理 C3
处理第 1 天
处理第 2 天
处理第 3 天
0:00:00.356046
处理 C4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.339071
处理 C5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.317151
处理 D1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.391976
处理 D2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.385940
处理 E1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:01.596706
处理 E2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:01.309523
处理 E3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天


In [26]:
mktind_backtest([(0,'',1)], [(15,4)], use_previous = turn_use_previous, lower=-0.11)

处理 A2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.769267
处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:02.031597
处理 A4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.334439
处理 A5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.381334
处理 B1
处理第 1 天
处理第 2 天
处理第 3 天
0:00:01.455080
处理 B3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.803175
处理 B4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.703471
处理 C2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.930851
处理 C3
处理第 1 天
处理第 2 天
处理第 3 天
0:00:01.862989
处理 C4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.888623
处理 C5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.824792
处理 D1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.871996
处理 D2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.913879
处理 E1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:03.405915
处理 E2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:03.472711
处理 E3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天


In [27]:
mktind_m_backtest([(0,'',1)], [(15,4)], use_previous = turn_use_previous, lower=-0.11)

处理 A2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.791210
处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:02.039569
处理 A4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.309523
处理 A5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.331438
处理 B1
处理第 1 天
处理第 2 天
处理第 3 天
0:00:01.496995
处理 B3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.809157
处理 B4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.685492
处理 C2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.774282
处理 C3
处理第 1 天
处理第 2 天
处理第 3 天
0:00:01.758293
处理 C4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.919512
处理 C5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.908597
处理 D1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.939809
处理 D2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:01.705437
处理 E1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:03.890882
处理 E2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
0:00:03.378961
处理 E3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天


###########################################################################################################################################

In [28]:
mktind_m_backtest([(0,'',1)], [(20,3)], key_list = ['H1','H2','H3','I1','I2','I3','I4','I5'], use_previous = turn_use_previous, lower=-0.11)

处理 H1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:04.390255
处理 H2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:04.002401
处理 H3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:04.692132
处理 I1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:06.227315
处理 I2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:06.545049
处理 I3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:06.005931
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理

In [29]:
mktind_backtest([(0,'',1)], [(20,3)], key_list = ['H1','H2','H3','I1','I2','I3','I4','I5'], use_previous = turn_use_previous, lower=-0.11)

处理 H1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:04.274037
处理 H2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:03.770569
处理 H3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
0:00:03.902084
处理 I1
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:04.761306
处理 I2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:05.471681
处理 I3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:05.220343
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理

###########################################################################################################################################

In [30]:
normal_backtest([(50000000,'5000万_',1), ], [30, 80, ], key_list = ['A3', ], use_previous = turn_use_previous, lower=-0.11)

处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.463793
处理 A3
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.424864


In [31]:
normal_backtest([(50000000,'5000万_',1), ], [30, ], key_list = ['C2', ], use_previous = turn_use_previous, lower=-0.11)

处理 C2
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.404954


In [32]:
normal_backtest([(50000000,'5000万_',1), ], [30, ], key_list = ['C4', ], use_previous = turn_use_previous, lower=-0.11)

处理 C4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
0:00:00.333110


In [33]:
normal_backtest([(50000000,'5000万_',1), ], [10, 60, 80, 150, ], key_list = ['I4', ], use_previous = turn_use_previous, lower=-0.11)
old_normal_backtest([(50000000,'5000万_',1), ], [10, 60, 80, 150, ], key_list = ['I4', ], use_previous = turn_use_previous, lower=-0.11)

处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.670446
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.373648
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.668830
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.469393
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.482686
处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 

In [34]:
normal_backtest([(50000000,'5000万_',1), ], [10, 30, 80, ], key_list = ['I5', ], use_previous = turn_use_previous, lower=-0.11)
old_normal_backtest([(50000000,'5000万_',1), ], [10, 30, 80, ], key_list = ['I5', ], use_previous = turn_use_previous, lower=-0.11)

处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.543016
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.834301
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.974454
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.768684
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
0:00:02.751034
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 

###########################################################################################################################################

#more data
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
hsIcData = pd.read_csv('hs300_from_db.csv')
hsIcData.columns = [int(i) for i in hsIcData.columns]
hsIcData[1] = [int(i) for i in hsIcData[1]]
hsIcData.set_index(1,inplace=True)
adjfactor = pd.read_csv('adjfactor.csv',index_col=0)
adjfactor.columns = [int(i) for i in adjfactor.columns]
hs300members = pd.read_csv('hs300members.csv',index_col=0).T
hs300members.index = [int(i) for i in hs300members.index]
total_shares = pd.read_csv('total_shares.csv',index_col=0)
total_shares.columns = [int(i) for i in total_shares.columns]
free_float_shares = pd.read_csv('free_float_shares.csv',index_col=0)
free_float_shares.columns = [int(i) for i in free_float_shares.columns]
adjusted_weight_coef = free_float_shares * close

hs300_backtest([(50000000,'5000万_',1)], [50], ['E1','E2','F1','H1','H2','H3','I1','I2','I3'], use_previous = False)

In [35]:
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
prest_kick = pd.read_csv('prest_kick_empty.csv',index_col = 0)
prest_kick.columns = [int(i) for i in prest_kick.columns]
prest_kick_normal_backtest([(50000000,'5000万s_',1)], [50], key_list=['I4','I5'], use_previous = False, quick_test=False)
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
prest_kick = pd.read_csv('new_prest_kick.csv',index_col = 0).loc[close.index].shift(1).dropna(how='all')
prest_kick.columns = [int(i) for i in prest_kick.columns]
prest_kick_normal_backtest([(50000000,'5000万n_',1)], [50], key_list=['I4','I5'], use_previous = False, quick_test=False)
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
prest_kick = pd.read_csv('prest_kick.csv',index_col = 0)
prest_kick.columns = [int(i) for i in prest_kick.columns]
prest_kick_normal_backtest([(50000000,'5000万o_',1)], [50], key_list=['I4','I5'], use_previous = False, quick_test=False)

处理 I4
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 38 天
处理第 39 天
处理第 40 天
处理第 41 天
处理第 42 天
处理第 43 天
处理第 44 天
处理第 45 天
处理第 46 天
处理第 47 天
处理第 48 天
处理第 49 天
处理第 50 天
处理第 51 天
处理第 52 天
处理第 53 天
处理第 54 天
处理第 55 天
处理第 56 天
处理第 57 天
处理第 58 天
处理第 59 天
处理第 60 天
处理第 61 天
处理第 62 天
处理第 63 天
处理第 64 天
处理第 65 天
处理第 66 天
处理第 67 天
处理第 68 天
处理第 69 天
处理第 70 天
处理第 71 天
处理第 72 天
处理第 73 天
处理第 74 天
处理第 75 天
处理第 76 天
处理第 77 天
处理第 78 天
处理第 79 天
处理第 80 天
处理第 81 天
处理第 82 天
处理第 83 天
处理第 84 天
处理第 85 天
处理第 86 天
处理第 87 天
处理第 88 天
处理第 89 天
处理第 90 天
处理第 91 天
处理第 92 天
处理第 93 天
处理第 94 天
处理第 95 天
处理第 96 天
处理第 97 天
处理第 98 天
处理第 99 天
处理第 100 天
处理第 101 天
处理第 102 天
处理第 103 天
处理第 104 天
处理第 105 天
处理第 106 天
处理第 107 天
处理第 108 天
处理第 109 天
处理第 110 天
处理

处理第 122 天
处理第 123 天
处理第 124 天
处理第 125 天
处理第 126 天
处理第 127 天
处理第 128 天
处理第 129 天
处理第 130 天
处理第 131 天
处理第 132 天
处理第 133 天
处理第 134 天
处理第 135 天
处理第 136 天
处理第 137 天
处理第 138 天
处理第 139 天
处理第 140 天
处理第 141 天
处理第 142 天
处理第 143 天
处理第 144 天
处理第 145 天
处理第 146 天
处理第 147 天
处理第 148 天
处理第 149 天
处理第 150 天
处理第 151 天
处理第 152 天
处理第 153 天
处理第 154 天
处理第 155 天
处理第 156 天
处理第 157 天
处理第 158 天
处理第 159 天
处理第 160 天
处理第 161 天
处理第 162 天
处理第 163 天
处理第 164 天
处理第 165 天
处理第 166 天
处理第 167 天
处理第 168 天
处理第 169 天
处理第 170 天
处理第 171 天
处理第 172 天
处理第 173 天
处理第 174 天
处理第 175 天
处理第 176 天
处理第 177 天
处理第 178 天
处理第 179 天
处理第 180 天
处理第 181 天
处理第 182 天
处理第 183 天
处理第 184 天
处理第 185 天
处理第 186 天
0:00:40.916553
处理 I5
处理第 1 天
处理第 2 天
处理第 3 天
处理第 4 天
处理第 5 天
处理第 6 天
处理第 7 天
处理第 8 天
处理第 9 天
处理第 10 天
处理第 11 天
处理第 12 天
处理第 13 天
处理第 14 天
处理第 15 天
处理第 16 天
处理第 17 天
处理第 18 天
处理第 19 天
处理第 20 天
处理第 21 天
处理第 22 天
处理第 23 天
处理第 24 天
处理第 25 天
处理第 26 天
处理第 27 天
处理第 28 天
处理第 29 天
处理第 30 天
处理第 31 天
处理第 32 天
处理第 33 天
处理第 34 天
处理第 35 天
处理第 36 天
处理第 37 天
处理第 3